<a href="https://colab.research.google.com/github/GISRS-academy/Google-earth-engine-google-colab/blob/main/Landcover%2C%20and%20LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ilacyEjIEQKig3wv1fpXcASCEcSlDbf_ryTra4it90A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjZ4h1ny7HMeSiarSRUbWD3PezN6z5nfg3whdv8IJr1nhgk9ccB-n8

Successfully saved authorization token.


In [2]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [9]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [11]:
# Define the urban location of interest as a point near Colombo, Sri Lanka.
xy = ee.Geometry.Point([79.90424801616612, 6.937342590385832])      



In [13]:
scale = 1000

# Print the elevation near Colombo, SL
elv_urban_point = elv.sample(xy, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(xy, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(xy, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Ground elevation at urban point: 8 m
Average daytime LST at urban point: 35.15 °C
Land cover value at urban point is: 13


In [14]:
# Define a region of interest with a buffer zone of 1000 km around colombo
roi = xy.buffer(1e6)

In [15]:
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

In [16]:
from IPython.display import Image

# Create a URL to the styled image for a region around colombo
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature in colombo
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8e129b63240829644e2601641e5530d0-2f9be15a9422528dcf50e7ff3ed15a61:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [27]:
# Make pixels with elevation below sea level transparent.
elv_img = elv.updateMask(elv.gt(0))

# Display the thumbnail of styled elevation in colombo
Image(url=elv_img.getThumbURL({
    'min': 0, 'max': 2000, 'dimensions': 512, 'region': roi,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))


In [28]:
# Create a buffer zone of 10 km around colombo
colombo = xy.buffer(10000)  # meters

url = elv_img.getThumbUrl({
    'min': 150, 'max': 350, 'region': colombo, 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [32]:
# Get a feature collection of administrative boundaries.
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

# Filter the feature collection to subset sri lanka 
srilanka = countries.filter(ee.Filter.eq('ADM0_NAME', 'Sri Lanka'))

# Clip the image by sri lanka 
elv_fr = elv_img.clip(srilanka )

# Create the URL associated with the styled image data.
url = elv_fr.getThumbUrl({
    'min': 0, 'max': 2500, 'region': roi, 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display a thumbnail of elevation in sri lanka 
Image(url=url)

In [33]:
task = ee.batch.Export.image.toDrive(image=elv_img,
                                     description='elevation_near_colombo',
                                     scale=30,
                                     region=colombo,
                                     fileNamePrefix='my_export_colombo',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [34]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1628870230134,
 'description': 'elevation_near_colombo',
 'destination_uris': ['https://drive.google.com/'],
 'id': '5UIILZFWFFFJF5EDPPTRBCF3',
 'name': 'projects/earthengine-legacy/operations/5UIILZFWFFFJF5EDPPTRBCF3',
 'start_timestamp_ms': 1628870250191,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1628870264159}

In [38]:
link = lst_img.getDownloadURL({
    'scale': 30,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': colombo})
print(link)
# Create the URL associated with the styled image data.
url = lst_img.getThumbUrl({
    'min': 15, 'max': 35, 'region': roi, 'dimensions': 512,
    'palette': ['yellow', 'green', 'red']})

# Display a thumbnail of LST in sri lanka 
Image(url=url)


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/36f2f4c2598a2585bde2a746564c3543-390b320ba4bf0bd2f661967bb2dc1130:getPixels


In [39]:
import folium

# Define the center of our map.
lat, lon =  6.937342590385832,79.90424801616612

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [41]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [42]:
# Select a specific band and dates for land cover.
lc_img = lc.select('LC_Type1').filterDate(i_date).first()

# Set visualization parameters for land cover.
lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

# Create a map.
lat, lon = 6.937342590385832,79.90424801616612
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(lc_img, lc_vis_params, 'Land Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [43]:
my_map.save('my_lc_interactive_map.html')

In [44]:
# Set visualization parameters for ground elevation.
elv_vis_params = {
    'min': 0, 'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Set visualization parameters for land surface temperature.
lst_vis_params = {
    'min': 0, 'max': 40,
    'palette': ['white', 'blue', 'green', 'yellow', 'orange', 'red']}

# Arrange layers inside a list (elevation, LST and land cover).
ee_tiles = [elv_img, lst_img, lc_img]

# Arrange visualization parameters inside a list.
ee_vis_params = [elv_vis_params, lst_vis_params, lc_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['Elevation', 'Land Surface Temperature', 'Land Cover']

# Create a new map.
lat, lon = 6.937342590385832,79.90424801616612
my_map = folium.Map(location=[lat, lon], zoom_start=5)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map